<a href="https://colab.research.google.com/github/carvalheiracarlos/deep_leaning_notebooks/blob/main/text_generation/twitter_generation_rnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install clean-text

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import pandas as pd
import tensorflow as tf
import numpy as np
import os
import time
from cleantext import clean

In [3]:
path_to_file = 'xorume_do_bolsonaro.txt'
# Read, then decode for py2 compat.
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
# length of text is the number of characters in it
print(f'Length of text: {len(text)} characters')

Length of text: 395857 characters


In [4]:
print(text[:250])

- É preciso compreender aqueles que ainda não decidiram e lhes oferecer segurança para que façam a melhor escolha para o futuro da nossa nação. Mais do que promessas vazias e abstratas, o Brasil precisa de um caminho sólido, pautado em ações concreta


In [5]:
text = clean(text, no_emoji=True, lower=False)
###text = text.apply(lambda x: x.encode('ascii', 'ignore').decode('ascii'))

In [6]:
vocab = sorted(set(text))
print(f'{len(vocab)} unique characters')
print(vocab[:250])

86 unique characters
['\n', ' ', '!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '=', '?', '@', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '|']


In [7]:
example_texts = ['abcdefg', 'xyz']

chars = tf.strings.unicode_split(example_texts, input_encoding='UTF-8')
chars

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>

In [8]:
# characters -> numbers (layer)
ids_from_chars = tf.keras.layers.StringLookup(
    vocabulary=list(vocab), mask_token=None)

In [9]:
# It converts from tokens to character IDs:
ids = ids_from_chars(chars)
ids

<tf.RaggedTensor [[60, 61, 62, 63, 64, 65, 66], [83, 84, 85]]>

In [10]:
# numbers -> ids (layer)
chars_from_ids = tf.keras.layers.StringLookup(
    vocabulary=ids_from_chars.get_vocabulary(), invert=True, mask_token=None)

In [11]:
# convert numbers -> characters
chars = chars_from_ids(ids)
chars

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>

In [12]:
# join characters back into text (layer)
tf.strings.reduce_join(chars, axis=-1).numpy()

array([b'abcdefg', b'xyz'], dtype=object)

In [13]:
# convert numbers into text
def text_from_ids(ids):
  return tf.strings.reduce_join(chars_from_ids(ids), axis=-1)

In [14]:
# text -> stream of indices (numbers)
all_ids = ids_from_chars(tf.strings.unicode_split(text, 'UTF-8'))
all_ids

<tf.Tensor: shape=(394214,), dtype=int64, numpy=array([15,  2, 37, ..., 68, 72, 64])>

In [15]:
ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids)
for ids in ids_dataset.take(10):
    print(chars_from_ids(ids).numpy().decode('utf-8'))

-
 
E
 
p
r
e
c
i
s


In [16]:
# size of the textual sequences to be used
seq_length = 100

sequences = ids_dataset.batch(seq_length+1, drop_remainder=True)

for seq in sequences.take(1):
  print(chars_from_ids(seq))

tf.Tensor(
[b'-' b' ' b'E' b' ' b'p' b'r' b'e' b'c' b'i' b's' b'o' b' ' b'c' b'o'
 b'm' b'p' b'r' b'e' b'e' b'n' b'd' b'e' b'r' b' ' b'a' b'q' b'u' b'e'
 b'l' b'e' b's' b' ' b'q' b'u' b'e' b' ' b'a' b'i' b'n' b'd' b'a' b' '
 b'n' b'a' b'o' b' ' b'd' b'e' b'c' b'i' b'd' b'i' b'r' b'a' b'm' b' '
 b'e' b' ' b'l' b'h' b'e' b's' b' ' b'o' b'f' b'e' b'r' b'e' b'c' b'e'
 b'r' b' ' b's' b'e' b'g' b'u' b'r' b'a' b'n' b'c' b'a' b' ' b'p' b'a'
 b'r' b'a' b' ' b'q' b'u' b'e' b' ' b'f' b'a' b'c' b'a' b'm' b' ' b'a'
 b' ' b'm' b'e'], shape=(101,), dtype=string)


In [17]:
for seq in sequences.take(5):
  print(text_from_ids(seq).numpy())

b'- E preciso compreender aqueles que ainda nao decidiram e lhes oferecer seguranca para que facam a me'
b'lhor escolha para o futuro da nossa nacao. Mais do que promessas vazias e abstratas, o Brasil precisa'
b' de um caminho solido, pautado em acoes concretas e, sobretudo, em principios.\n- Logo mais, as 19h, f'
b'aremos um importante anuncio em nossas redes sociais. Peco que compartilhem este convite com todos os'
b' seus contatos para que o nosso anuncio chegue a todos os brasileiros. Voltamos em breve!\n- Diferente'


In [18]:
# For training you'll need a dataset of (input, label) pairs.
# Where input and label are sequences.
# At each time step the input is the current character and the label is the next character.
# Here's a function that takes a sequence as input, duplicates, and shifts it
# to align the input and label for each timestep:
def split_input_target(sequence):
    input_text = sequence[:-1]
    target_text = sequence[1:]
    return input_text, target_text

In [19]:
dataset = sequences.map(split_input_target)

for input_example, target_example in dataset.take(1):
    print("Input :", text_from_ids(input_example).numpy())
    print("Target:", text_from_ids(target_example).numpy())

Input : b'- E preciso compreender aqueles que ainda nao decidiram e lhes oferecer seguranca para que facam a m'
Target: b' E preciso compreender aqueles que ainda nao decidiram e lhes oferecer seguranca para que facam a me'


In [20]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = (
    dataset
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE))

dataset

<PrefetchDataset element_spec=(TensorSpec(shape=(64, 100), dtype=tf.int64, name=None), TensorSpec(shape=(64, 100), dtype=tf.int64, name=None))>

In [21]:
# Length of the vocabulary in StringLookup Layer
vocab_size = len(ids_from_chars.get_vocabulary())

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [22]:
# building the model
class MyModel(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, rnn_units):
    super().__init__(self)
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(rnn_units,
                                   return_sequences=True,
                                   return_state=True)
    self.dense = tf.keras.layers.Dense(vocab_size)

  def call(self, inputs, states=None, return_state=False, training=False):
    x = inputs
    x = self.embedding(x, training=training)
    if states is None:
      states = self.gru.get_initial_state(x)
    x, states = self.gru(x, initial_state=states, training=training)
    x = self.dense(x, training=training)

    if return_state:
      return x, states
    else:
      return x

In [23]:
# load model
model = MyModel(
    vocab_size=vocab_size,
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)

In [24]:
# trying the model
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 87) # (batch_size, sequence_length, vocab_size)


In [25]:
model.summary()

Model: "my_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       multiple                  22272     
                                                                 
 gru (GRU)                   multiple                  3938304   
                                                                 
 dense (Dense)               multiple                  89175     
                                                                 
Total params: 4,049,751
Trainable params: 4,049,751
Non-trainable params: 0
_________________________________________________________________


In [26]:
# To get actual predictions from the model you need to sample from the output distribution,
#to get actual character indices. This distribution is defined by the logits over the character vocabulary.
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices, axis=-1).numpy()

In [27]:
# example
sampled_indices

array([82, 52, 61, 75, 30, 84, 44, 49, 52, 86,  7,  7, 78, 26, 51, 29, 39,
        5,  7, 76, 74, 54, 75,  2, 18, 51, 50, 59, 83,  5, 47, 46, 47, 20,
       78, 56, 55, 13, 71, 56, 46, 72, 72, 16,  8, 69, 30, 50,  1, 14,  4,
        7, 18,  5, 13, 80, 19, 23, 50,  1, 71, 71, 51, 15, 18, 60, 67,  2,
        3, 39, 72, 54, 71, 60, 67, 84, 19, 47, 85, 75, 11, 79, 51,  8, 17,
       11, 45, 78, 85, 11, 55, 20, 28, 73, 14, 76, 73, 44, 15, 28])

In [28]:
# Decode these to see the text predicted by this untrained model:
print("Input:\n", text_from_ids(input_example_batch[0]).numpy())
print()
print("Next Char Predictions:\n", text_from_ids(sampled_indices).numpy())

Input:
 b' em mal estar no mercado gerado pelo governo. Pois bem, a Bolsa de Valores atingiu hoje mais uma mar'

Next Char Predictions:
 b'wTbp=yLQT|%%s8S;G#%qoVp 0SR_x#ONO2sXW+lXNmm.&j=R\n,"%0#+u15R\nllS-0ah !GmVlahy1Ozp)tS&/)Msz)W2:n,qnL-:'


In [29]:
# cost function
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)

In [30]:
# cost (untrained)
example_batch_mean_loss = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("Mean loss:        ", example_batch_mean_loss)

Prediction shape:  (64, 100, 87)  # (batch_size, sequence_length, vocab_size)
Mean loss:         tf.Tensor(4.4662104, shape=(), dtype=float32)


In [31]:
tf.exp(example_batch_mean_loss).numpy()

87.0263

In [32]:
# model compilation
model.compile(optimizer='adam', loss=loss)

In [33]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [34]:
EPOCHS = 20

In [35]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/20
60/60 [==============================] - 5s 54ms/step - loss: 3.2327
Epoch 2/20
60/60 [==============================] - 4s 58ms/step - loss: 2.4659
Epoch 3/20
60/60 [==============================] - 5s 59ms/step - loss: 2.2745
Epoch 4/20
60/60 [==============================] - 4s 54ms/step - loss: 2.1417
Epoch 5/20
60/60 [==============================] - 4s 52ms/step - loss: 2.0174
Epoch 6/20
60/60 [==============================] - 3s 51ms/step - loss: 1.8916
Epoch 7/20
60/60 [==============================] - 4s 53ms/step - loss: 1.7717
Epoch 8/20
60/60 [==============================] - 4s 52ms/step - loss: 1.6571
Epoch 9/20
60/60 [==============================] - 4s 52ms/step - loss: 1.5606
Epoch 10/20
60/60 [==============================] - 4s 53ms/step - loss: 1.4767
Epoch 11/20
60/60 [==============================] - 4s 54ms/step - loss: 1.4040
Epoch 12/20
60/60 [==============================] - 4s 53ms/step - loss: 1.3369
Epoch 13/20
60/60 [==================

In [36]:
# text generation model
class OneStep(tf.keras.Model):
  def __init__(self, model, chars_from_ids, ids_from_chars, temperature=1.0):
    super().__init__()
    self.temperature = temperature
    self.model = model
    self.chars_from_ids = chars_from_ids
    self.ids_from_chars = ids_from_chars

    # Create a mask to prevent "[UNK]" from being generated.
    skip_ids = self.ids_from_chars(['[UNK]'])[:, None]
    sparse_mask = tf.SparseTensor(
        # Put a -inf at each bad index.
        values=[-float('inf')]*len(skip_ids),
        indices=skip_ids,
        # Match the shape to the vocabulary
        dense_shape=[len(ids_from_chars.get_vocabulary())])
    self.prediction_mask = tf.sparse.to_dense(sparse_mask)

  @tf.function
  def generate_one_step(self, inputs, states=None):
    # Convert strings to token IDs.
    input_chars = tf.strings.unicode_split(inputs, 'UTF-8')
    input_ids = self.ids_from_chars(input_chars).to_tensor()

    # Run the model.
    # predicted_logits.shape is [batch, char, next_char_logits]
    predicted_logits, states = self.model(inputs=input_ids, states=states,
                                          return_state=True)
    # Only use the last prediction.
    predicted_logits = predicted_logits[:, -1, :]
    predicted_logits = predicted_logits/self.temperature
    # Apply the prediction mask: prevent "[UNK]" from being generated.
    predicted_logits = predicted_logits + self.prediction_mask

    # Sample the output logits to generate token IDs.
    predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
    predicted_ids = tf.squeeze(predicted_ids, axis=-1)

    # Convert from token ids to characters
    predicted_chars = self.chars_from_ids(predicted_ids)

    # Return the characters and model state.
    return predicted_chars, states

In [37]:
one_step_model = OneStep(model, chars_from_ids, ids_from_chars)

In [38]:
start = time.time()
states = None
next_char = tf.constant(['Bolsonaro:'])
result = [next_char]

for n in range(256):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)
print('\nRun time:', end - start)

Bolsonaro: Visando interna estimada de R$ 2,9 bilhoes em irresponsaveis! O cargo de recursos pelas outupros da falta de assentou na Nota transa apreendeu mais de R$1 mil pessoas q 16.000 vez e 11,4% no COMPROV.ANO POSTO c/ @TorrinCAJHHJ)
- @minsaude ha varios escand 

________________________________________________________________________________

Run time: 1.4281296730041504
